In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise
from scipy.sparse import csr_matrix
import sklearn
import pickle

In [6]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [7]:
with open('./distance_recommender.pkl', 'rb') as file:
    model = pickle.load(file)

In [14]:
def recommend_neighborhood(query, model, ratings, k=10):

    user_vec = np.repeat(0, 168253)
    user_vec[list(query.keys())] = list(query.values())

    distances, userIds = model.kneighbors([user_vec], n_neighbors=10, return_distance=True)

    distances = distances[0]
    userIds = userIds[0]

    neighborhood = ratings.set_index('userId').loc[userIds]

    factors = np.array(distances)[neighborhood.index.factorize()[0]]
    neighborhood['rating'] *= 1-factors
    
    scores = neighborhood.groupby('movieId')['rating'].sum()
    

    allready_seen = scores.index.isin(query.keys())
    scores.loc[allready_seen] = 0
    scores = scores.sort_values(ascending=False)

    recommendations = scores.head(k).index

    result = movies.set_index('movieId').loc[recommendations]

    return result

In [15]:
test_query = {1: 5, 20: 5, 3: 5, 4: 5, 5: 5, 7:5}

In [16]:
recommend_neighborhood(test_query, model, ratings, k = 10)

,title,genres
movieId,,
780,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical
648,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller
32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
25,Leaving Las Vegas (1995),Drama|Romance
733,"Rock, The (1996)",Action|Adventure|Thriller
141,"Birdcage, The (1996)",Comedy
95,Broken Arrow (1996),Action|Adventure|Thriller
736,Twister (1996),Action|Adventure|Romance|Thriller
